In [ ]:
import pandas as pd
import ast
import re
import glob
import numpy as np
import plotly.express as px
import plotly
import sarracen
import os
import k3d
import ipywidgets
import matplotlib.pyplot as plt

Here, we're using [`sarracen`](https://sarracen.readthedocs.io/en/latest/examples/dustydisc.html) for dump file visualization.

### Importing Dump Files

In [ ]:
# Define the parent directory
parent_dir = "/home/adm61595/CHLab/1_HCA_PPDs/1_Code/1_Notebooks/Exp17/dumpfiles" # Replace depending on path

In [ ]:
def gatherdumps(parent_dir):
    sims = {}      # Dictionary to store indexed sdf files
    simsinks = {}  # Dictionary to store indexed sdf_sinks files

    # Iterate over all subdirectories in the parent directory
    for idx in range(100):  # From 00 to 99
        sim_folder = parent_dir
        target_file = os.path.join(sim_folder, "dustysgdisc_00020")

        # Check if the target file exists in the directory
        if os.path.isfile(target_file):
            # Read the file using sarracen.read_phantom
            sdf, sdf_sinks = sarracen.read_phantom(target_file)
            
            # Store in dictionaries with indexed keys
            sims[f"sdf_{idx}"] = sdf
            simsinks[f"sdf_sinks_{idx}"] = sdf_sinks
            print(f"Loaded: {target_file} as sdf_{idx} and sdf_sinks_{idx}")

        else:
            print(f"{target_file} not found in {sim_folder}.")

    return sims, simsinks

In [ ]:
sims, simsinks = gatherdumps(parent_dir)

Making each dump file accessible from their individual name:

In [ ]:
sdfs = []
sdf_sinks = []

for key, sdf in sims.items():
    globals()[key] = sdf
    sdfs.append(sdf)

for key, sdf_sink in simsinks.items():
    globals()[key] = sdf_sink
    sdf_sinks.append(sdf_sink)

In [ ]:
display(sdf_0)

### Visualizing Outputs in 2D

In [ ]:
dimax=200


In [ ]:

for idx, sdf in enumerate(sdfs):
    plt.figure()  # Create a new figure for each `sdf`
    
    # Render with logarithmic scaling and automatic color range
    sdf.render(
        'rho',
        xlim=(-dimax, dimax),
        ylim=(-dimax, dimax),
        xsec=0.0,
        cmap='inferno'
    )
    plt.title(f'SDF Plot {idx + 1}')  # Optional: add title for each plot
    plt.show()  # Display the plot

In [ ]:
dimax = 80
sdf_39.render('rho', xlim=(-dimax, dimax), ylim=(-dimax, dimax), log_scale=False, xsec=0.0)

In [ ]:
ax = sdf_39.render('rho', xlim=(-dimax, dimax), ylim=(-dimax, dimax), log_scale=False, xsec=0.0)
ax.scatter(x=sdf_sinks_39['x'], y=sdf_sinks_39['y'], color='white')

In [ ]:
sdf_39.sph_interpolate('rho')

### Visualizing Outputs in 3D with K3D

Calculating the density for the gas component:

In [ ]:
sdf_39.calc_density()

To extract the positions and densities of the SPH particles:

In [ ]:
gas_39_pos   = np.dstack((sdf_39.x,sdf_39.y,sdf_39.z))
sinks_39_pos = np.dstack((sdf_sinks_39.x,sdf_sinks_39.y,sdf_sinks_39.z))

In [ ]:
gas_39_dens = np.log10(sdf_39.rho)

In [ ]:
 dens_39_interp = sdf_39.sph_interpolate('rho',
                                  x_pixels=200,
                                  y_pixels=200,
                                  z_pixels=200,
                                  xlim=(-300,300),
                                  ylim=(-300,300),
                                  zlim=(-300,300),
                                  )

In [ ]:
gas_39_particles = k3d.points(
    gas_39_pos.astype(np.float32),
    attribute=gas_39_dens,
    name='Gas particles',
    shader='3d',
    point_size=0.1
    )

In [ ]:
sink_39_particles = k3d.points(
    sinks_39_pos.astype(np.float32),
    name='Sink particles',
    shader='3d',
    color=0xffffff,
    point_size=3,
    )

In [ ]:
volume_39 = k3d.volume(
    dens_39_interp.astype(np.float32),
    name='Gas mesh',
    bounds=[-300, 300, -300, 300, -300, 300]
    )

In [ ]:
plot = k3d.plot()

In [ ]:
plot += gas_39_particles
plot += sink_39_particles
plot += volume_39

In [ ]:
plot.background_color = 0x000000
plot.grid_visible = False

Displaying doesn't work because of some jupyter notebook issue, but we can download and visualize this!

In [ ]:
with open('snapshot.html', 'w') as f:
    f.write(plot.get_snapshot())

### Visualizing the Orszag-Tang Vortex

In [ ]:
sdf_39.render('rho', xsec=0.0)

In [ ]:
sdf_39.streamlines(('x', 'y', 'z'), xsec=0.0)

In [ ]:
sdf_39['P'] = sdf_39['u'] * sdf_39['rho'] * (sdf_39.params['gamma'] - 1.0)

sdf_39.lineplot('P', xlim=(-0.5, 0.5), ylim=-0.1875, zlim=0.0)